### Lesson 12 Bonus Lab:  Gradient Boosting With the Bikeshare Dataset

Welcome!  This notebook is designed to provide additional practice for people looking to get more familiar with Gradient Boosting & SciKit Learn.

The topic of the notebook is using Gradient Boosting to forecast demand for bikeshare stations.

The dataset has the following columns:  

  - **datetime:** a timestamp collected hourly.
  - **season:** a categorical column that lists the current season for that observation
  - **holiday:** a column (0 or 1), that detects whether or not it was a holiday
  - **workingday:** a column (0 or 1), that encodes whether or not it was a workday or not
  - **weather:** a categorical column that lists a light weather description for the observation
  - **temp:** the temperature outside
  - **atemp:** the temperature it feels like outside
  - **humidity:** the humidity outside
  - **windspeed:** the windspeed, in mph
  - **count:** the number of bikes checked out during that hour
  
Your job is to build a regression model that appropriately captures the information available to make the most accurate predictions.

### Step 1:  Load in the Dataset

 - It's called `bikeshare.csv`
 - Make sure to make `datetime` a time column
 - It's not a bad idea to use it as an index column, although this isn't necessary.

In [4]:
# your answer here
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor

df = pd.read_csv('../../data/bikeshare.csv', parse_dates=['datetime'])

### Step 2: Transform Your Categorical Variables (If Necessary)

This dataset has two categorical columns -- `weather` and `season`.  Decide how you might encode these -- One Hot encoding, ordinal encoding, or categorical encoding (or try several if you have time).

In [7]:
# your answer here
cat_cols = df.select_dtypes(include=np.object).columns.tolist()
df[cat_cols] = df[cat_cols].astype('category')
for col in cat_cols:
    df[col] = df[col].cat.codes

### Step 3: Create a Training & Test Set

Given that there's a time based column, make the most recent values your test set.  Do a 20% split.  (You can use `train_test_split` for this, but it's not necessary.  You could also just sort by `datetime` and take the bottom 20% of rows for your test set).

**Note:** You can use the argument `shuffle=False` if you want to use `train_test_split` without shuffling the data.

In [16]:
# your answer here
X = df.drop(['count', 'datetime'], axis=1)
y = df['count']

In [17]:
X_train, X_test, y_train, y_test = X[:8000].copy(), X[8000:].copy(), y[:8000].copy(), y[8000:].copy()

### Step 4: Create a Validation Set From Your Training Set

Remember....this is your test set within the training set.  Make it 20% of your training set.

In [18]:
# your answer here
X_train, X_val, y_train, y_val = X_train[:6400].copy(), X_train[6400:].copy(), y_train[:6400].copy(), y_train[6400:].copy()

### Step 5:  Do An Initial Fitting And Scoring of Your Model

 - Remember, fit on the training set, and score on the validation set.
 - How much is your model overfitting (if at all)?

In [20]:
# your answer here
gbm = GradientBoostingRegressor()
gbm.fit(X_train, y_train)
gbm.score(X_val, y_val)

0.10766459062257538

### Step 6: Look At Your Feature Importance Scores

What seems to be having the most impact?

In [21]:
# your answer here
feats = pd.DataFrame({
    'Features': X.columns,
    'Importances': gbm.feature_importances_
}).sort_values(by='Importances', ascending=False)

feats.head()

,Features,Importances
4,temp,0.290335
6,humidity,0.289762
5,atemp,0.262040
2,workingday,0.056994
0,season,0.051021


### Step 7: Build New Features (ie, Add New Columns To Your Dataset)

This is your chance to think about ways to better capture the value and impact of time and other variables on the target variable (`count`).

What you should do here is add a new feature to your training and validation set, re-run your model on the  training set, and score it on the validation set to see if it made an improvement.  

A good place to start with this is extracting out different date parts to see if they improve your validation score.

You can find information about the different dateparts in pandas here:  https://pandas.pydata.org/pandas-docs/version/0.24/reference/series.html#time-series-related

Or if you're using the `datetime` column as an index:  https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DatetimeIndex.html

Keep features if they improve your validation score, discard them if they don't.

A few other ideas:  

 - you can create a column called `Daytime` that tests whether or not it's light outside.  (Ie, between 7PM - 7AM is `False`, `True` otherwise).  You could also get fancier and adjust the daylight hours depending on season.  

 - you can also create a variable that tracks the passage of time.  This can be done by finding the earliest date in the dataset, subtracting each observed date from that and extracting the datepart in days.  This way, if you have an upward or downward trend throughout the dataset, you'd be able to capture it. So something like `X_train['time'] = (X_train.index.hour - earliest_date).days` 
 - You could also try multiplying different columns together.  Maybe it being `Daytime`, `Sunny` and low humidity has a multiplicative effect that isn't totally captured by any of the variables by themselves.

**Note:** Dateparts, despite being numbers, are probably best thought of as **categorical** variables.....think about it -- the 11 AM hour is something distinct from the 11 PM hour.....they are best interpreted as being separate categories than one continuous column.

In [51]:
# your answer here
# we'll add in the datetime column back into our dataset and refit
X['datetime'] = df['datetime']
X['hour'] = X['datetime'].dt.hour
X['day']   = X['datetime'].dt.day
X['month'] = X['datetime'].dt.month
X['quarter'] = X['datetime'].dt.quarter
X['time'] = (X['datetime'] - X['datetime'].min()).dt.days
X['daytime'] = np.where((X['hour'] > 19) & (X['hour'] < 8), True, False)
X.drop('datetime', axis=1, inplace=True)

In [52]:
X_train, X_test, y_train, y_test = X[:8000].copy(), X[8000:].copy(), y[:8000].copy(), y[8000:].copy()
X_train, X_val, y_train, y_val = X_train[:6400].copy(), X_train[6400:].copy(), y_train[:6400].copy(), y_train[6400:].copy()

In [53]:
gbm.fit(X_train, y_train)
gbm.score(X_val, y_val)

0.7872793165940405

In [47]:
# parameter scoring
num_trees = [100, 500, 1000, 5000]
learning_rates = [.001, .01, .1]
max_depth = [2, 3, 4]
cv_scores = []

for tree in num_trees:
    for rate in learning_rates:
        for depth in max_depth:
            print(f"New round of model fitting.  Testing params:  num_trees: {tree}, learning_rate: {rate}, max_depth: {depth}")
            gbm.set_params(n_estimators=tree, learning_rate=rate, max_depth=depth)
            gbm.fit(X_train, y_train)
            mod_score = gbm.score(X_val, y_val)
            print(f"Score this round: {mod_score}")
            cv_scores.append((mod_score, tree, rate, depth))

New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.001, max_depth: 2
Score this round: -0.206620946948727
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.001, max_depth: 3
Score this round: -0.19445975393114834
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.001, max_depth: 4
Score this round: -0.18455106971758561
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.01, max_depth: 2
Score this round: 0.0960298028956702
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.01, max_depth: 3
Score this round: 0.16661796232319082
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.01, max_depth: 4
Score this round: 0.2430993989410897
New round of model fitting.  Testing params:  num_trees: 100, learning_rate: 0.1, max_depth: 2
Score this round: 0.4836991119137151
New round of model fitting.  Testing params:  num_trees: 100, 

In [48]:
# our max score
max(cv_scores)

(0.786334132795435, 500, 0.1, 4)

In [49]:
# we'll set our model to these parameters
gbm.set_params(n_estimators=500, learning_rate=0.1, max_depth=4)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [57]:
# we'll look at feature importances as well
feats = pd.DataFrame({
    'Features': X.columns,
    'Importances': gbm.feature_importances_
}).sort_values(by='Importances', ascending=False)

feats

,Features,Importances
10,hour,0.386736
8,hours,0.252088
2,workingday,0.094127
13,time,0.085805
5,atemp,0.068591
4,temp,0.053244
6,humidity,0.036526
3,weather,0.011226
7,windspeed,0.004387
9,day,0.002836


In [56]:
X.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hours,day,hour,month,quarter,time,daytime
0,1,0,0,0,9.84,14.395,81,0.0,0,1,0,1,1,0,False
1,1,0,0,0,9.02,13.635,80,0.0,1,1,1,1,1,0,False
2,1,0,0,0,9.02,13.635,80,0.0,2,1,2,1,1,0,False
3,1,0,0,0,9.84,14.395,75,0.0,3,1,3,1,1,0,False
4,1,0,0,0,9.84,14.395,75,0.0,4,1,4,1,1,0,False


### Step 8: Fit Your Model On ALL of your training data

An important step here -- now that you've figured out what columns to include, and what ones to exclude, concatenate your training and validation sets, and fit your model on ALL of your training data.

The idea now is that you've found the features that help, you should give your model more samples to infer from.

You would use the `pd.concat()` method here.

Also -- for good measure, standardize all of your training data before fitting it if you haven't done so already.

In [58]:
# your answer here
# we'll create the total training set here:
X_train = pd.concat([X_train, X_val])
y_train = pd.concat([y_train, y_val])

gbm.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Step 9: Score Your Model on the Test Set

Once you've found the best version of your model on your validation set, transform your test set so that it is setup the same way.

Ie, if you added a column that improved your validation score, add that same column to your test set.  

Remember to standardize your test set using the values from your training set.

How close were your validation scores to your test set scores?

In [59]:
# your answer here
gbm.score(X_test, y_test)

0.8651051069897694

### Diagnostics

Now we'll look at a few different areas of your model to see if there's anything causing our results to be skewed.

### Step 10:  Make a prediction on your test set, and calculate the error

The error in this case is just the difference between the value for `count` and the value of your prediction.

In [60]:
# your answer here
df['Prediction'] = gbm.predict(X)

In [61]:
df['Error'] = df['count'] - df['Prediction']